<a href="https://colab.research.google.com/github/mohanrajmit/Transfer-Learning/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from pathlib import Path
import numpy as np
import joblib

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [22]:
!git clone https://github.com/mohanrajmit/Transfer-Learning.git

fatal: destination path 'Transfer-Learning' already exists and is not an empty directory.


In [23]:
!ls

bird_feature_classifier_model.h5  sample_data  Transfer-Learning  x_train.dat  y_train.dat


In [24]:
# Load the training data set by looping over every image file
for image_file in Path("Transfer-Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224, 224))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [25]:
print(np.array(x_train).shape)

(2000, 224, 224, 3)


In [26]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = imagenet_utils.preprocess_input(x_train)



In [27]:

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))



In [28]:
feature_extractor_resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [29]:
feature_extractor.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [30]:
feature_extractor_resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [31]:
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)



63/63 [==============================] - 8s 124ms/step


In [32]:
features_x.shape

(2000, 7, 7, 512)

In [33]:
features_resnet = feature_extractor_resnet.predict(x_train)

63/63 [==============================] - 9s 103ms/step


In [34]:
features_resnet.shape

(2000, 7, 7, 2048)

In [11]:
# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

In [35]:
Y_train = np.array(y_train)
print(x_train.shape[1:])
print(Y_train.shape)

(224, 224, 3)
(2000,)


In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [37]:
# Create a model_vgg and add layers
model_vgg = Sequential()

# Add layers to our model_vgg
model_vgg.add(Flatten(input_shape=features_x.shape[1:]))
model_vgg.add(Dense(128, activation='relu'))
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(1, activation='sigmoid'))

# Compile the model_vgg
model_vgg.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [38]:
# Create a model_vgg and add layers
model_resnet = Sequential()

# Add layers to our model_vgg
model_resnet.add(Flatten(input_shape=features_resnet.shape[1:]))
model_resnet.add(Dense(128, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(1, activation='sigmoid'))

# Compile the model_vgg
model_resnet.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [39]:
# Train the model_vgg
model_vgg.fit(features_x,Y_train,epochs=10,shuffle=True, verbose=2)

# Save the trained model_vgg to a file so we can use it to make predictions later
model_vgg.save("bird_feature_classifier_model_vgg.h5")

Epoch 1/10
63/63 - 1s - loss: 1.6671 - accuracy: 0.7175 - 992ms/epoch - 16ms/step
Epoch 2/10
63/63 - 0s - loss: 0.3875 - accuracy: 0.8270 - 253ms/epoch - 4ms/step
Epoch 3/10
63/63 - 0s - loss: 0.3010 - accuracy: 0.8610 - 253ms/epoch - 4ms/step
Epoch 4/10
63/63 - 0s - loss: 0.2638 - accuracy: 0.8705 - 255ms/epoch - 4ms/step
Epoch 5/10
63/63 - 0s - loss: 0.2272 - accuracy: 0.8975 - 251ms/epoch - 4ms/step
Epoch 6/10
63/63 - 0s - loss: 0.1976 - accuracy: 0.9130 - 272ms/epoch - 4ms/step
Epoch 7/10
63/63 - 0s - loss: 0.1615 - accuracy: 0.9290 - 254ms/epoch - 4ms/step
Epoch 8/10
63/63 - 0s - loss: 0.1406 - accuracy: 0.9350 - 253ms/epoch - 4ms/step
Epoch 9/10
63/63 - 0s - loss: 0.1291 - accuracy: 0.9380 - 275ms/epoch - 4ms/step
Epoch 10/10
63/63 - 0s - loss: 0.1021 - accuracy: 0.9565 - 245ms/epoch - 4ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
# Train the model_vgg
model_resnet.fit(features_resnet,Y_train,epochs=10,shuffle=True, verbose=2)

# Save the trained model_vgg to a file so we can use it to make predictions later
model_resnet.save("bird_feature_classifier_model_resnet.h5")

Epoch 1/10
63/63 - 1s - loss: 5.2998 - accuracy: 0.7400 - 1s/epoch - 23ms/step
Epoch 2/10
63/63 - 1s - loss: 0.2936 - accuracy: 0.8805 - 792ms/epoch - 13ms/step
Epoch 3/10
63/63 - 1s - loss: 0.2186 - accuracy: 0.9095 - 794ms/epoch - 13ms/step
Epoch 4/10
63/63 - 1s - loss: 0.1929 - accuracy: 0.9150 - 773ms/epoch - 12ms/step
Epoch 5/10
63/63 - 1s - loss: 0.1334 - accuracy: 0.9425 - 801ms/epoch - 13ms/step
Epoch 6/10
63/63 - 1s - loss: 0.1246 - accuracy: 0.9495 - 662ms/epoch - 11ms/step
Epoch 7/10
63/63 - 1s - loss: 0.1184 - accuracy: 0.9545 - 629ms/epoch - 10ms/step
Epoch 8/10
63/63 - 1s - loss: 0.0843 - accuracy: 0.9615 - 625ms/epoch - 10ms/step
Epoch 9/10
63/63 - 1s - loss: 0.0849 - accuracy: 0.9615 - 631ms/epoch - 10ms/step
Epoch 10/10
63/63 - 1s - loss: 0.0895 - accuracy: 0.9620 - 629ms/epoch - 10ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [40]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [41]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224,224,3))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = imagenet_utils.preprocess_input(x_test)

In [42]:
x_test.shape

(200, 224, 224, 3)

In [43]:
# Load our trained classifier model_vgg
model_vgg = load_model("bird_feature_classifier_model_vgg.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model_vgg
predictions = model_vgg.predict(features_x)

# If the model_vgg is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model_vgg makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

7/7 [==============================] - 0s 3ms/step
True Positives: 86
True Negatives: 83
False Positives: 17
False Negatives: 14
              precision    recall  f1-score   support

           0       0.86      0.83      0.84       100
           1       0.83      0.86      0.85       100

    accuracy                           0.84       200
   macro avg       0.85      0.84      0.84       200
weighted avg       0.85      0.84      0.84       200



In [45]:
# Load our trained classifier model_vgg
model_resnet = load_model("bird_feature_classifier_model_resnet.h5")

# Extract features for each image (all in one pass)
features_resnet = feature_extractor_resnet.predict(x_test)

# Given the extracted features, make a final prediction using our own model_vgg
predictions = model_resnet.predict(features_resnet)

# If the model_vgg is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model_vgg makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

7/7 [==============================] - 0s 5ms/step
True Positives: 88
True Negatives: 84
False Positives: 16
False Negatives: 12
              precision    recall  f1-score   support

           0       0.88      0.84      0.86       100
           1       0.85      0.88      0.86       100

    accuracy                           0.86       200
   macro avg       0.86      0.86      0.86       200
weighted avg       0.86      0.86      0.86       200

